In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import time
import logging
import sys
import copy
import gc
# draw metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torchvision.models as models
from torchvision.models import AlexNet_Weights
import torchmetrics

import torchvision.transforms as transforms
from torchvision.transforms import Compose 
from torch.utils.tensorboard import SummaryWriter
torch.backends.cudnn.benchmark=True
print(sys.executable)
print(sys.version)
print(torch.cuda.is_available())
print(f'Torchvision version: {torchvision.__version__}')

c:\Users\jizhi\.conda\envs\flv1\python.exe
3.8.18 (default, Sep 11 2023, 13:39:12) [MSC v.1916 64 bit (AMD64)]
True
Torchvision version: 0.16.0


In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_value = 42
set_seed(seed_value)

In [3]:
# set logger
# create logger
logger = logging.getLogger(__name__)
# set log level for all handlers to debug
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
# best for development or debugging
consoleHandler = logging.StreamHandler()
consoleHandler.setLevel(logging.DEBUG)

# create formatter
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s-%(levelname)s: %(message)s')

# add formatter to ch
consoleHandler.setFormatter(formatter)

# add ch to logger
logger.addHandler(consoleHandler)

In [4]:
##### Hyperparameters for federated learning #########
epochs_ = 50
batch_size_ = 16
writer_ = SummaryWriter('runs\\sim-fusion-alexnet\\')

# CUDA_LAUNCH_BLOCKING=1

In [5]:
# prepare image
train_names_list = ['hospital','sim-room','warehouse']

val_names_list = ['hospital','sim-room','warehouse','overall']

train_data_folders_list = [ 'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\for_Centered_learning\\hospital_sim-room\\train',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\for_Centered_learning\\hospital_warehouse\\train',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\for_Centered_learning\\sim-room_warehouse\\train',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\for_Centered_learning\\hospital_sim-room_warehouse\\train',
                    ]

val_data_folders_list = [ 'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\hospital\\test',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\sim-room\\test',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\warehouse\\test',
                            'C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_data\\mine\\sim_train\\overall\\test',
                    ]

transformed_train_datasets, transformed_val_datasets = [], []
################################################  Data Loaders  #####################################
# 在这个部分，为训练和验证的图像数据定义了一些预处理步骤，包括颜色抖动、重新调整大小、转换为张量和归一化。
# 这些预处理步骤通过 PyTorch 的 transforms 模块实现，并且它们将在训练过程中应用到每个图像上。
for train_dataset in train_data_folders_list:
    transformed_train_datasets.append(
        datasets.ImageFolder(
            train_dataset,
            transforms.Compose([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
    ) 

for val_dataset in val_data_folders_list:
    transformed_val_datasets.append(
        datasets.ImageFolder(
            val_dataset,
            transforms.Compose([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
    ) 

#########  Data Loaders  ##########

train_loader_list, val_loader_list = [], []

for train_d in transformed_train_datasets:
    train_loader_list.append(
        torch.utils.data.DataLoader(
                train_d,
                batch_size=batch_size_,
                shuffle=True,
                num_workers=4,
                drop_last = False,
            )
    )

for val_d in transformed_val_datasets:
    val_loader_list.append(
        torch.utils.data.DataLoader(
                val_d,
                batch_size=1,
                shuffle=True,
                num_workers=4,
                drop_last = False,
            )
    )

In [6]:
def test(model_name, writer, current_epoch, model, test_loader):
    """
    This function test the global model on test 
    data and returns test loss and test accuracy 
    """
    model.eval()

    #test_loss = 0
    #correct = 0
    test_error_count1 = 0
    with torch.no_grad():
        for image, label in test_loader:#以batch为单位取值
            image, label = image.cuda(), label.cuda()
            outputs = model(image)

            test_error_count1 += float(torch.sum(torch.abs(label - outputs.argmax(1))))
        # 用于计算测试阶段的平均损失
        #test_loss /= len(test_loader.dataset)


        acc = 1.0 - float(test_error_count1) / float(len(test_loader.dataset))
        writer.add_scalar("Val-Acc/"+model_name, acc, current_epoch)

    return acc

In [7]:
model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)

for param in model.parameters():
    param.requires_grad = False

# 这行代码修改了AlexNet模型分类器的最后一层。它将最后一层的输出特征数从原始的1000（AlexNet在ImageNet上的分类数）改为了2。
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

device = torch.device('cuda')

model = model.to(device)

optims_=optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


logger.info("-----------------------Start--------------------------")

#这行代码创建了一个梯度缩放器，它通常用于半精度（float16）训练，这可以加速训练并减少GPU内存使用。如果你不打算使用半精度训练，这个部分可以忽略。
scaler = torch.cuda.amp.grad_scaler.GradScaler()

best_acc = 0.0
for i in range(epochs_):
    for images, labels in train_loader_list[0]:
        images = images.to(device)
        labels = labels.to(device)
        optims_.zero_grad()
        with torch.cuda.amp.autocast_mode.autocast():
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optims_)
        scaler.update()

    val_inx  = 3 #用overall数据集来决定最后存储的模型
    val_data = val_loader_list[val_inx]

    acc = test("Fusion_Models:hospital_sim-room-On-Val-Data-{}".format(val_names_list[val_inx]), writer_, i, model,  val_data)
    logger.info("Centered Learning Model:hospital_sim-room, Epoch:{}, Val_Data:{}, Accuracy:{}.".format(i, val_names_list[val_inx], acc))

    if acc > best_acc and acc < 1.0:
        best_acc = acc
        save_path = f"C:\\Users\\jizhi\\Desktop\\study\\FL\\federated_learning\\fl_results\\saved_models\\sim\\fusion_models\\Centered_Learning_hospital_sim-room_center.pth"
        torch.save(model.state_dict(), save_path)
logger.info("Final Centered Learning Model:hospital_sim-room, Best Accuracy:{}.".format(best_acc))

logger.info("DONE!!!!!!!!")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\jizhi/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:21<00:00, 11.4MB/s] 
2023-11-02 18:26:54,163-INFO: -----------------------Start--------------------------
2023-11-02 18:27:38,540-INFO: Centered Learning Model:hospital_sim-room, Epoch:0, Val_Data:overall, Accuracy:0.9735576923076923.
2023-11-02 18:28:21,713-INFO: Centered Learning Model:hospital_sim-room, Epoch:1, Val_Data:overall, Accuracy:0.8774038461538461.
2023-11-02 18:29:04,070-INFO: Centered Learning Model:hospital_sim-room, Epoch:2, Val_Data:overall, Accuracy:0.9014423076923077.
2023-11-02 18:29:46,316-INFO: Centered Learning Model:hospital_sim-room, Epoch:3, Val_Data:overall, Accuracy:0.8774038461538461.
2023-11-02 18:30:28,268-INFO: Centered Learning Model:hospital_sim-room, Epoch:4, Val_Data:overall, Accuracy:0.8677884615384616.
2023-11-02 18:31:09,978-INFO: Centered Learning Model:hospital_sim-roo